In [5]:
%load_ext autoreload
%autoreload 2

from models.ranked_transformer import Moonshot
import torch, pytorch_lightning as pl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
from models.chemformer.tokeniser import MolEncTokeniser
from models.chemformer.utils import REGEX


path = "tempdata/chemformer/moonshot.ckpt"
token_file = "tempdata/chemformer/bart_vocab.txt"

tokeniser = MolEncTokeniser.from_vocab_file(
  token_file, REGEX, 272
)
model = Moonshot.load_from_checkpoint(path, tokeniser=tokeniser).cuda()


Initialized SignCoordinateEncoder[512] with dims [224, 224, 64] and 2 positional encoders. 64 bits are reserved for encoding the final bit





Ranked Transformer callin super


In [7]:
from utils.init_utils.data_init import data_mux


feats = ["HSQC", "SMILES"]
feat_handlers = ["pad", "tokenise"]
ds_path = "tempdata/SMILES_dataset/"

gim = data_mux(feats, feat_handlers, ds_path, token_file = token_file)

In [8]:
gim.setup("fit")
val_dl = gim.val_dataloader()

In [9]:
from rdkit import Chem, RDLogger
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import Draw

RDLogger.DisableLog("rdApp.*")

In [10]:
ct = 0
choice = ""
import random


pl.utilities.seed.isolate_rng(1234)

for batch in val_dl:
  hsqc, collated_smiles = batch
  hsqc = hsqc.cuda()
  collated_smiles = {k:(v if type(v) is not torch.Tensor else v.cuda()) for k,v in collated_smiles.items()}
  out = model.sample_rdm(hsqc, temperature=1, gen_len = 128)
  raw = collated_smiles["raw_smiles"]
  output_strs = out["mol_strs"]
  print(len(output_strs))
  for idx, (r, o) in enumerate(zip(raw, output_strs)):
    print(torch.sum(hsqc[idx]), Chem.MolFromSmiles(o) is not None, len(o), r, o)
  ct += 1
  if ct == 1:
    break

Max seq len: 128


100%|██████████| 127/127 [00:02<00:00, 58.94it/s]


32
tensor(9239.9336, device='cuda:0') False 2 c1(O)ccc2c(c1O)-c1c3c(cc(O)c1)CCN(C)C3C2.Br c)
tensor(25262.3242, device='cuda:0') False 95 c1c(Br)c2cc[nH]c2cc1Br c-c1(()=cc2COc)ccccc))(cc(1)(1)-)cc3C11C2c(Occ1)1(c=1cc1OPcCc4cOco1OcBrOccc(cO((1)c)CcOc)c1)O)O
tensor(47867.4492, device='cuda:0') False 22 c12ccc(Br)cc1c1ccncc1[nH]2 (cnccc22)1c1))2cBr(cc3
tensor(35896.0391, device='cuda:0') False 27 c1c(Br)cc2c([nH]cc2)c1 Oc)Cc2C1)cCc21c)3cCcCccOccC
tensor(-1252.0664, device='cuda:0') False 63 C(C#CC#CC=CCO)#C )C(O=()()(OC=()((CCCCCC=(cc)=)5c)C)=)=(OC=Cc)C)CO1=CCC)CCC2OCC1
tensor(26917.6543, device='cuda:0') False 43 O1C(=O)Cc2ccccc2C1C#CC#C OcNcCO)1(2c(OO)COOO)(O(c)C1c=COc1(1)2()(c(c
tensor(-474954.9375, device='cuda:0') False 52 OC(C#CC#CCO)CCCCCCCCCCCC=CCCCCCCCCCC=CC(=O)C#C cCCCCCC)CCCCCOOCC)CCC(Cc2OCCCCCC#)cCOCccCcC)CCOCC)Cc
tensor(-103315.5547, device='cuda:0') False 34 C(C(C#C)O)=CCCCCCCCCCCC=CC(C#C)=O CCC(C#CCCCCCCC#)=CCOCCCC#CCC(CC)C)
tensor(21518.8164, device='cuda:0') False 37 C1(B

In [11]:
v = Chem.MolFromSmiles("(CCcCOCc4CCCCCC(5cCCC=CCCc2CCCC)=1C=C=CC)")
print(v)

None
